In [2]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier

import warnings
warnings.filterwarnings('ignore')

#### Load Symptom Dataset

In [3]:
data = pd.read_csv('./Data/final.csv', sep = ',')

####  Load Symptom Severity 

In [4]:
severity = pd.read_csv('./Data/Symptom-severity.csv')

####  Load Exam Mapping

In [5]:
mapping = pd.read_csv('./Data/map.csv', sep = ';')

# Split data for train and test

In [6]:
symptomCount = 131
testCount = 102

[train,test] = train_test_split(data,random_state=420)
trainA = np.array(train)
trainAX = trainA[:,testCount:]
trainAY = trainA[:,:testCount]

trainX=train.iloc[:,testCount:]
trainY=train.iloc[:,:testCount]
#print(trainY[0:5])

testX=test.iloc[:,testCount:]
testY=test.iloc[:,:testCount]

In [7]:
trainY
encodedY=[]
maxx=0
for line in trainY.to_numpy():
    encList = int("".join(str(int(x)) for x in line), 2)
    maxx = max(maxx,len(str(encList)))
    encodedY.append(encList)
#print(maxx)
#print(len(trainY.to_numpy()[0]))
arra=[]
for line in trainY.to_numpy():
    arra.append(line)
#print(arra)
np.unique(np.array(arra))

array([0, 1])

In [8]:
encodingMap= dict()
betterEncodedY=[]
for i in encodedY:
    if(not i in encodingMap):
        encodingMap[i]= str(i)
    betterEncodedY.append(encodingMap[i])


# SVM

In [9]:
clf = OneVsRestClassifier(SVC(kernel='poly',degree=2, gamma=0.5, C=0.1)).fit(trainX.to_numpy(), trainY.to_numpy())

In [10]:
def realAccuracies(predictions, labels):
    thresh =(predictions.max()+ predictions.min())/2
    predictions[predictions>thresh]=1
    predictions[predictions<=thresh]=0
    
    correctByLine=[]
    for i in range(len(testY)):
        matches = predictions[i]==labels[i]
        correctByLine.append(len(matches[matches==True])/len(matches))
        
    return([len(np.nonzero([x for x in correctByLine if x==1])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.99])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.95])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.90])[0]),
            len(np.nonzero([x for x in correctByLine if x!=1])[0])])


def getRealAcc(clf):
    predicted = clf.predict(testX)
    accs =  realAccuracies(predicted, testY.to_numpy())

    print(accs[3]," classified 90% correctly")
    print(accs[2]," classified 95% correctly")
    print(accs[1]," classified 99% correctly")
    print(accs[0]," classified 100% correctly")
    print(accs[4]," classified incorrectly (Not 100%)")

In [11]:
def gridSearch(model, param_grid, X_train, y_train, X_test, y_test):        
    clf = GridSearchCV(model, param_grid, refit=True, verbose=3)
    clf.fit(X_train.to_numpy(),y_train.to_numpy())
    print(clf.best_params_)
    predictions = clf.predict(testX)
    getRealAcc(clf)
    return ;

In [12]:
param_grid_svc = {
    'class_weight': ['balanced', None], 
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001],
     'kernel': ['rbf', 'poly', 'linear']
} 

In [13]:
svc = OneVsRestClassifier(SVC().fit(trainX.to_numpy(), trainY.to_numpy()))
gridSearch(svc, param_grid_svc, trainX, trainY.to_numpy(), testX, testY)

ValueError: y should be a 1d array, got an array of shape (3510, 102) instead.

In [14]:
clf = OneVsRestClassifier(SVC(kernel='poly',degree=2, gamma=0.5, C=0.1)).fit(trainX.to_numpy(), trainY.to_numpy())

In [15]:
def actualAccuracies( predictions, labels):
    thresh =(predictions.max()+ predictions.min())/2
    predictions[predictions>thresh]=1
    predictions[predictions<=thresh]=0
    
    correctByLine=[]
    for i in range(len(testY)):
        matches = predictions[i]==labels[i]
        correctByLine.append(len(matches[matches==True])/len(matches))
    return([len(np.nonzero([x for x in correctByLine if x==1])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.99])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.95])[0]),
            len(np.nonzero([x for x in correctByLine if x>0.90])[0]),
            len(np.nonzero([x for x in correctByLine if x!=1])[0])
           ])

In [16]:
preds = clf.predict(testX)
accs =  actualAccuracies(preds, testY.to_numpy())
print(accs[3]," classified 90% correctly")
print(accs[2]," classified 95% correctly")
print(accs[1]," classified 99% correctly")
print(accs[0]," classified 100% correctly")
print(accs[4]," classified incorrectly (Not 100%)")

1170  classified 90% correctly
1170  classified 95% correctly
1170  classified 99% correctly
1170  classified 100% correctly
0  classified incorrectly (Not 100%)
